In [58]:
import pandas as pd
import numpy as np
import numpy_financial as npf

from datetime import date, datetime, timedelta

# AIMS

1. Ratio de endeudamiento
2. Suma de cuota mensual monetaria por cada credito
3. Suma de creditos per month
4. Razon de uso de credito

## 1. Ratio de endeudamiento por cliente general

In [59]:
creditos = pd.read_csv('creditos.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
creditos.head()

,id_cliente,id_prestamo,monto,cuotas,deuda_actual,fecha_inicio,fecha_pago,fecha_pago.1,dias_atraso
0,1,10001,100,10,80,12/12/2022,12/12/2022,12/12/2022,0
1,1,10002,200,5,150,22/12/2022,22/12/2022,23/12/2022,1
2,2,10003,50,10,5,23/12/2022,23/12/2022,28/12/2022,5
3,2,10004,75,3,70,24/12/2022,24/12/2022,28/12/2022,4
4,2,10005,125,5,12,24/12/2022,24/12/2022,29/12/2022,5


In [60]:
clientes_agrupados = creditos.groupby('id_cliente').sum()
clientes_agrupados

C:\Users\Oscar\AppData\Local\Temp\ipykernel_4780\2360885637.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  clientes_agrupados = creditos.groupby('id_cliente').sum()


,id_prestamo,monto,cuotas,deuda_actual,dias_atraso
id_cliente,,,,,
1,40017,650,22,241,4
2,40019,650,28,109,18


In [61]:
clientes_agrupados['ratio_endeudamiento'] = clientes_agrupados['deuda_actual'] / clientes_agrupados['monto']
clientes_agrupados

,id_prestamo,monto,cuotas,deuda_actual,dias_atraso,ratio_endeudamiento
id_cliente,,,,,,
1,40017,650,22,241,4,0.370769
2,40019,650,28,109,18,0.167692


## 2. Suma de cuota mensual monetaria por cada credito

In [62]:
# funcion mes
def get_month(x):
    return datetime(x.year, x.month, 1)

In [63]:
creditos = pd.DataFrame({
    "Cliente": ["Juan", "Juan", "Ana", "Ana", "Pedro", "Pedro"],
    "Monto": [1000, 5000, 2000, 3000, 1500, 2000],
    "Cuotas": [12, 24, 6, 12, 8, 12],
    "Fecha": ["2022-01-01", "2022-02-01", "2022-01-01", "2022-03-01", "2022-01-01", "2022-02-01"]
})
creditos["Fecha"] = pd.to_datetime(creditos["Fecha"])
creditos['fecha_mes'] = creditos['Fecha'].apply(get_month)
creditos.set_index("fecha_mes", inplace=True)
creditos

,Cliente,Monto,Cuotas,Fecha
fecha_mes,,,,
2022-01-01,Juan,1000,12,2022-01-01
2022-02-01,Juan,5000,24,2022-02-01
2022-01-01,Ana,2000,6,2022-01-01
2022-03-01,Ana,3000,12,2022-03-01
2022-01-01,Pedro,1500,8,2022-01-01
2022-02-01,Pedro,2000,12,2022-02-01


In [64]:
creditos["Cuota Mensual"] = creditos["Monto"] / creditos["Cuotas"]
creditos

,Cliente,Monto,Cuotas,Fecha,Cuota Mensual
fecha_mes,,,,,
2022-01-01,Juan,1000,12,2022-01-01,83.333333
2022-02-01,Juan,5000,24,2022-02-01,208.333333
2022-01-01,Ana,2000,6,2022-01-01,333.333333
2022-03-01,Ana,3000,12,2022-03-01,250.000000
2022-01-01,Pedro,1500,8,2022-01-01,187.500000
2022-02-01,Pedro,2000,12,2022-02-01,166.666667


In [65]:
deuda_mensual = creditos.groupby([pd.Grouper(freq="M"), "Cliente"])["Cuota Mensual"].sum()
deuda_mensual

fecha_mes   Cliente
2022-01-31  Ana        333.333333
            Juan        83.333333
            Pedro      187.500000
2022-02-28  Juan       208.333333
            Pedro      166.666667
2022-03-31  Ana        250.000000
Name: Cuota Mensual, dtype: float64

In [66]:
deuda_mensual = deuda_mensual.reset_index()
deuda_mensual

,fecha_mes,Cliente,Cuota Mensual
0,2022-01-31,Ana,333.333333
1,2022-01-31,Juan,83.333333
2,2022-01-31,Pedro,187.500000
3,2022-02-28,Juan,208.333333
4,2022-02-28,Pedro,166.666667
5,2022-03-31,Ana,250.000000


## 3. Suma de creditos per month

In [67]:
# out = (creditos.assign(month=creditos['date'].dt.to_period('M'))
#         # duplicate rows corresponding to the duration
#         .loc[creditos.index.repeat(creditos['duration'])]
#         # increment months and ensure correct order
#         .assign(month=lambda d: d['month']+d.groupby(level=0).cumcount().add(1))
#         .sort_values(by=['id_customer', 'month'])
#         # cumulated sum of installments
#         .assign(total_sum=lambda d: d.groupby(level=0)['installment'].cumsum())
#         # sum per customer
#         .groupby(['id_customer', 'month'], as_index=False)
#         .agg(**{'total_sum': ('total_sum', 'sum'),
#                 'sum_cred': ('credit', 'sum'),
#                })
#         # computation of total_inv and shift of sum_cred
#         .assign(total_inv=lambda d: d['sum_cred'].sub(d['total_sum']),
#                 sum_cred=lambda d: d.groupby('id_customer')['sum_cred'].shift(-1, fill_value=0)
#                )
#        )

# out

In [68]:
creditos = pd.DataFrame({
    "id_customer": ["A", "B", "A"],
    "id_credit":[1001, 1002, 1003],
    "duration": [4,3,5],
    "date": ["2022-01-02", "2022-02-03", "2022-03-04"],
    "credit": [100, 90, 250],
    "installment": [25,30,50],
})
creditos['installment_amount'] = round(creditos['credit'] / creditos['installment'],2)
creditos["date"] = pd.to_datetime(creditos["date"])
creditos

,id_customer,id_credit,duration,date,credit,installment,installment_amount
0,A,1001,4,2022-01-02,100,25,4.0
1,B,1002,3,2022-02-03,90,30,3.0
2,A,1003,5,2022-03-04,250,50,5.0


In [69]:
## Crea una columna llamada month en la cual se da entender el mes de toma del credito
creditos = creditos.assign(month=creditos['date'].dt.to_period('M'))
creditos

,id_customer,id_credit,duration,date,credit,installment,installment_amount,month
0,A,1001,4,2022-01-02,100,25,4.0,2022-01
1,B,1002,3,2022-02-03,90,30,3.0,2022-02
2,A,1003,5,2022-03-04,250,50,5.0,2022-03


In [70]:
## Repite la fila por la duracion, posteriormente le adiciona un mes por linea
creditos = creditos.loc[creditos.index.repeat(creditos['duration'])].assign(month=lambda d: d['month']+d.groupby(level=0).cumcount().add(1))
creditos

,id_customer,id_credit,duration,date,credit,installment,installment_amount,month
0,A,1001,4,2022-01-02,100,25,4.0,2022-02
0,A,1001,4,2022-01-02,100,25,4.0,2022-03
0,A,1001,4,2022-01-02,100,25,4.0,2022-04
0,A,1001,4,2022-01-02,100,25,4.0,2022-05
1,B,1002,3,2022-02-03,90,30,3.0,2022-03
1,B,1002,3,2022-02-03,90,30,3.0,2022-04
1,B,1002,3,2022-02-03,90,30,3.0,2022-05
2,A,1003,5,2022-03-04,250,50,5.0,2022-04
2,A,1003,5,2022-03-04,250,50,5.0,2022-05
2,A,1003,5,2022-03-04,250,50,5.0,2022-06


In [71]:
## Ordena el dataframe por customer and month
creditos = creditos.sort_values(by=['id_customer', 'month'])
creditos

,id_customer,id_credit,duration,date,credit,installment,installment_amount,month
0,A,1001,4,2022-01-02,100,25,4.0,2022-02
0,A,1001,4,2022-01-02,100,25,4.0,2022-03
0,A,1001,4,2022-01-02,100,25,4.0,2022-04
2,A,1003,5,2022-03-04,250,50,5.0,2022-04
0,A,1001,4,2022-01-02,100,25,4.0,2022-05
2,A,1003,5,2022-03-04,250,50,5.0,2022-05
2,A,1003,5,2022-03-04,250,50,5.0,2022-06
2,A,1003,5,2022-03-04,250,50,5.0,2022-07
2,A,1003,5,2022-03-04,250,50,5.0,2022-08
1,B,1002,3,2022-02-03,90,30,3.0,2022-03


In [72]:
## Crea la columna total_sum
creditos = creditos.assign(total_sum=lambda d: d.groupby(level=0)['installment'].cumsum())
creditos

,id_customer,id_credit,duration,date,credit,installment,installment_amount,month,total_sum
0,A,1001,4,2022-01-02,100,25,4.0,2022-02,25
0,A,1001,4,2022-01-02,100,25,4.0,2022-03,50
0,A,1001,4,2022-01-02,100,25,4.0,2022-04,75
2,A,1003,5,2022-03-04,250,50,5.0,2022-04,50
0,A,1001,4,2022-01-02,100,25,4.0,2022-05,100
2,A,1003,5,2022-03-04,250,50,5.0,2022-05,100
2,A,1003,5,2022-03-04,250,50,5.0,2022-06,150
2,A,1003,5,2022-03-04,250,50,5.0,2022-07,200
2,A,1003,5,2022-03-04,250,50,5.0,2022-08,250
1,B,1002,3,2022-02-03,90,30,3.0,2022-03,30


In [73]:
## Agrupa por customer and month
creditos = creditos.groupby(['id_customer', 'month'], as_index=False).agg(**{'total_sum': ('total_sum', 'sum'),
                                                                             'sum_cred': ('credit', 'sum'),})
creditos

,id_customer,month,total_sum,sum_cred
0,A,2022-02,25,100
1,A,2022-03,50,100
2,A,2022-04,125,350
3,A,2022-05,200,350
4,A,2022-06,150,250
5,A,2022-07,200,250
6,A,2022-08,250,250
7,B,2022-03,30,90
8,B,2022-04,60,90
9,B,2022-05,90,90


In [74]:
creditos = creditos.assign(total_inv=lambda d: d['sum_cred'].sub(d['total_sum']),
                           sum_cred=lambda d: d.groupby('id_customer')['sum_cred'].shift(-1, fill_value=0)
                          )
creditos

,id_customer,month,total_sum,sum_cred,total_inv
0,A,2022-02,25,100,75
1,A,2022-03,50,350,50
2,A,2022-04,125,350,225
3,A,2022-05,200,250,150
4,A,2022-06,150,250,100
5,A,2022-07,200,250,50
6,A,2022-08,250,0,0
7,B,2022-03,30,90,60
8,B,2022-04,60,90,30
9,B,2022-05,90,0,0


## 4. Razon de uso de credito

In [75]:
creditos

,id_customer,month,total_sum,sum_cred,total_inv
0,A,2022-02,25,100,75
1,A,2022-03,50,350,50
2,A,2022-04,125,350,225
3,A,2022-05,200,250,150
4,A,2022-06,150,250,100
5,A,2022-07,200,250,50
6,A,2022-08,250,0,0
7,B,2022-03,30,90,60
8,B,2022-04,60,90,30
9,B,2022-05,90,0,0


In [76]:
creditos2 = creditos.groupby('id_customer')['sum_cred'].max().reset_index()
creditos2

,id_customer,sum_cred
0,A,350
1,B,90


In [83]:
# redondear a multiplo de 10 es con -1,  100 es con -2, 1000 es con -3
creditos2['suma_redondeada'] = round(creditos2['sum_cred'], -2)
creditos2

,id_customer,sum_cred,suma_redondeada
0,A,350,400
1,B,90,100
